In [1]:
import pandas as pd 

import pandas as pd

from sklearn.model_selection import train_test_split

from kmodes.kmodes import KModes
from sklearn.model_selection import train_test_split
from pycaret.classification import *

%run class/patient_details.py
%run util/utils.py

Cardiovascular Disease Predictor

In [2]:
#Cardiovascular
cardio_df = pd.read_csv("data/cardio_clean.csv")

df_male = cardio_df.query("gender == 1")
df_female = cardio_df.query("gender == 0")

# female data
# Building the model with using K-Mode with "Huang" initialization
km_huang = KModes(n_clusters=2, init = "Huang", n_init = 5, verbose=0)

male_kmode = km_huang.fit(df_male)
female_kmode = km_huang.fit(df_female)

clusters_female = km_huang.fit_predict(df_female)
clusters_male = km_huang.fit_predict(df_male)

# female DataFrame with Clusters
df_female.insert(0,"cluster", clusters_female, True)

# male DataFrame with Clusters
df_male.insert(0, "cluster", clusters_male, True)

# replacing cluster column values to merge dataframes after
df_male["cluster"].replace({0:2, 1:3}, inplace=True)

# merging female and male data
df_clusters = pd.concat([df_female, df_male], ignore_index=True, sort=False)

clf1 = setup(data = df_clusters, 
             target = 'cardio',
             preprocess = False,
             silent = True)

models()

top5 = compare_models(sort='AUC',
                    n_select = 5,
                 )
stack_model = stack_models(estimator_list = top5, meta_model = top5[0],optimize = 'AUC')

final_model = finalize_model(stack_model)

def predict_cvd(patient):
    return final_model.predict(patient)

def assign_cluster_male(patient):
    cluster = male_kmode.predict(patient)
    if cluster[0] == 0: return 2
    if cluster[0] == 1: return 3 

def assign_cluster_female(patient):
    predict = female_kmode.predict(patient)
    return predict[0]


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8641,0.9476,0.8424,0.8740,0.8579,0.7278,0.7283
1,0.8525,0.9415,0.8244,0.8662,0.8448,0.7044,0.7052
2,0.8572,0.9443,0.8195,0.8791,0.8483,0.7138,0.7154
3,0.8696,0.9504,0.8459,0.8815,0.8633,0.7387,0.7393
4,0.8684,0.9492,0.8351,0.8880,0.8607,0.7362,0.7375
5,0.8770,0.9527,0.8561,0.8873,0.8714,0.7535,0.7540
6,0.8610,0.9492,0.8352,0.8740,0.8542,0.7216,0.7223
7,0.8727,0.9508,0.8654,0.8722,0.8688,0.7452,0.7452
8,0.8632,0.9456,0.8445,0.8708,0.8574,0.7260,0.7263


In [3]:
def assign_cluster_male(patient):
    cluster = male_kmode.predict(patient)
    if cluster[0] == 0: return 2
    if cluster[0] == 1: return 3 

def assign_cluster_female(patient):
    predict = female_kmode.predict(patient)
    return predict[0]

df_male.head()
x = assign_cluster_male([[1, 3, 1, 0, 0, 1, 1, 1, 1]])
y = assign_cluster_female([[1, 3, 1, 0, 0, 1, 1, 1, 1]])

Diabetes Predictor

In [4]:
#Diabetes
diabetes_df = pd.read_csv('data/diabetes_clean.csv')
diabetes_df.info()

X = diabetes_df.iloc[:,:-1]
y = diabetes_df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

clf1 = setup(data = diabetes_df, 
             target = 'Outcome',
             preprocess = False,
             silent = True)
             
models()

top5 = compare_models(sort='AUC', n_select = 5)

blend_soft = blend_models(estimator_list = top5, optimize = 'AUC',method = 'soft')

cali_model = calibrate_model(blend_soft)

final_model = finalize_model(cali_model)

def predict_diabetes(patient):
    prediction = final_model.predict(patient)
    return prediction[0]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7963,0.8406,0.6316,0.7500,0.6857,0.5367,0.5410
1,0.6667,0.6992,0.2632,0.5556,0.3571,0.1692,0.1908
2,0.8148,0.8992,0.6316,0.8000,0.7059,0.5735,0.5820
3,0.7593,0.8586,0.6316,0.6667,0.6486,0.4658,0.4661
4,0.8148,0.8857,0.7368,0.7368,0.7368,0.5940,0.5940
5,0.7963,0.8511,0.5263,0.8333,0.6452,0.5123,0.5389
6,0.7222,0.7248,0.4737,0.6429,0.5455,0.3520,0.3605
7,0.7358,0.8429,0.5000,0.6429,0.5625,0.3775,0.3836
8,0.7358,0.8460,0.6667,0.6000,0.6316,0.4266,0.4280


Web Application Implementation

In [6]:
%run class/patient_details.py
%run util/utils.py

patient = Patient(cluster = 1, 
                  age = 30,
                  gender = 1, 
                  glucose = 100, 
                  systolic_bp = 120, 
                  diastolic_bp = 80, 
                  mean_arterial_pressure =  90,
                  skin_thickness = 20, 
                  insulin = 34, 
                  bmi = 22, 
                  dpf = 1, 
                  age_bin = 1, 
                  bmi_class = 1, 
                  map_class = 1, 
                  cholesterol_class = 1, 
                  glucose_class = 1, 
                  smoke = 0, 
                  active = 1)

print_string("Padme")

Padme
